In [106]:
import json
import webbrowser
import os
import pandas as pd
from random import randrange

class No:
    
    def __init__(self, nome, conteudo = None):
        self.nome = nome
        self.arestas = []
        self.conteudo = conteudo
        self.tipo = 'vazio'
        self.df_imagens_presa = pd.read_csv('imagens_presa.csv')
        self.df_imagens_cacador = pd.read_csv('imagens_cacador.csv')
        self.df_imagens_cidade = pd.read_csv('imagens_cidades.csv')
        
    def adicionar_aresta(self, aresta):
        self.arestas.append(aresta)
        
    def setar_no_com_cacador(self):
        self.tipo = 'cacador'
        
    def setar_no_com_presa(self):
        self.tipo = 'presa'
        
    def get_json(self):
        
        no_json = {"id": self.nome, "label": self.nome, "shape": "circularImage", "image": self.get_imagem_aleatoria()}
            
        return no_json
    
    def get_imagem_aleatoria(self):
        
        if self.tipo == 'presa':
            idx = randrange(self.df_imagens_presa.shape[0])
            return self.df_imagens_presa['link'].tolist()[idx]
        
        elif self.tipo == 'cacador':
            idx = randrange(self.df_imagens_cacador.shape[0])
            return self.df_imagens_cacador['link'].tolist()[idx]
        
        else:
            idx = randrange(self.df_imagens_cidade.shape[0])
            return self.df_imagens_cidade['link'].tolist()[idx]
            
        
        
class Aresta:
    
    def __init__(self, de, para, peso):
        self.de = de
        self.para = para
        self.peso = peso
        
    def get_json(self):
        return {"from": self.de, "to": self.para, "label": str(self.peso), "arrows": {"to": {"enabled": True}}}
        
class Grafo:
    
    def __init__(self):
        self.nos = []
        self.json_nos = []
        self.json_arestas = []
    
    def inicializar_estrutura(self, numero_cidades, numero_estradas , numero_membros):
        
        self.numero_cidades  = numero_cidades
        self.numero_estradas = numero_estradas
        self.numero_membros  = numero_membros
        
        for i in range(self.numero_cidades):
            self.nos.append(No(i))
            
    def adicionar_aresta(self, de, para, peso):
        aresta = Aresta(de, para, peso)
        self.nos[de].adicionar_aresta(aresta)
        self.nos[para].adicionar_aresta(aresta)
        self.json_arestas.append(aresta.get_json())
        
    def setar_no_com_presa(self, no):
        self.nos[no].setar_no_com_presa()
        
    def setar_no_com_cacador(self, no):
        self.nos[no].setar_no_com_cacador()
                
    def coletar_json_nos(self):
        for no in self.nos:
            self.json_nos.append(no.get_json())
            
    def printar_grafo(self):
        
        self.coletar_json_nos()
        
        html = '''
            <!DOCTYPE html>
            <html lang="pt-br">
              <head>
                <title>Grafo</title>

                <style type="text/css">

                  #mynetwork {
                    width: 100%;
                    height: 100%;
                    position: absolute; 
                    top: 0; left: 0;
                  }
                </style>

                <script
                  type="text/javascript"
                  src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"

                ></script>

                <script type="text/javascript">

                  var nos = '''+ json.dumps(self.json_nos) +''';
                  var arestas = '''+ json.dumps(self.json_arestas) +''';
                  var network = null;


                  function draw() {
                    nodes = '''+ json.dumps(self.json_nos) +''';

                    // Instantiate our network object.
                    var container = document.getElementById("mynetwork");
                    var data = {
                      nodes: nos,
                      edges: arestas,
                    };
                    var options = {
                      physics: { enabled: true },
                      nodes: {
                        

                      },
                    };
                    network = new vis.Network(container, data, options);

                    var net = document.getElementById('mynetwork');


                  }
                </script>
              </head>
              <body onload="draw()">
                <div id="mynetwork"></div>
              </body>
            </html>
        '''
        

        # salvando o grafo localmente
        filename = "grafo.html"
        f = open(filename, "w")
        f.write(html)
        f.close()
        # abrindo no navegador
        webbrowser.open('file://' + os.path.realpath(filename))
        
        
class ConstrutorDeGrafo:
    
    def construir_grafo (self,caminho_arquivo_txt):
        g = Grafo()

        f = open(caminho_arquivo_txt, "r")
        line = f.readline()
        counter = 0
        counter_presa = 0
        while(line):
            line = line.split('\n')[0]
            array = line.split(' ')
            array = [int(i) for i in array]

            if counter == 0:
                numero_cidades  = array[0]
                numero_estradas = array[1]
                numero_membros  = array[2]

                g.inicializar_estrutura(numero_cidades, numero_estradas , numero_membros)

            elif len(array) == 3:
                de   = array[0]
                para = array[1]
                peso = array[2]
                g.adicionar_aresta(de, para, peso)

            elif counter_presa < numero_membros:
                no =  array[0]
                g.setar_no_com_presa(no)

                counter_presa+=1 
            else:
                no =  array[0]
                g.setar_no_com_cacador(no)

            line = f.readline()
            counter+=1

        f.close()
        
        return g

In [113]:
entrada = 'entradas\\Entrada5.txt'
g = ConstrutorDeGrafo().construir_grafo(entrada)

In [114]:
g.printar_grafo()